In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import zarr
import dask
import distributed
import os #needed to check that file exists
from dask.distributed import Client, progress


print('xarray: %s-%s' % (xr.__version__, xr.__file__))
print('dask: %s-%s' % (dask.__version__, dask.__file__))
print('distributed: %s-%s' % (distributed.__version__, distributed.__file__))
print('zarr: %s-%s' % (zarr.__version__, zarr.__file__))

#def get_sat_filename(date):
#    dir_sat='F:/data/sst/jpl_mur/v4.1/'
#    date = pd.to_datetime(date)
#    syr, smon, sdym, sjdy = str(date.year), str(date.month).zfill(2), str(date.day).zfill(2), str(date.dayofyear).zfill(2)
#    sat_filename =dir_sat + syr + '/'+ sjdy + '/' + syr + smon + sdym + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
#    exists = os.path.isfile(sat_filename)
#    return sat_filename, exists


xarray: 0.12.1-C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\__init__.py
dask: 1.2.0-C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\dask\__init__.py
distributed: 1.27.1-C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\distributed\__init__.py
zarr: 2.3.1-C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\zarr\__init__.py


In [2]:
xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.7.1 | packaged by conda-forge | (default, Mar 13 2019, 13:32:59) [MSC v.1900 64 bit (AMD64)]
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 63 Stepping 2, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None
libhdf5: 1.10.4
libnetcdf: 4.6.2

xarray: 0.12.1
pandas: 0.24.2
numpy: 1.16.3
scipy: 1.2.1
netCDF4: 1.5.1.1
pydap: None
h5netcdf: None
h5py: 2.9.0
Nio: None
zarr: 2.3.1
cftime: 1.0.3.4
nc_time_axis: None
PseudonetCDF: None
rasterio: 1.0.22
cfgrib: None
iris: None
bottleneck: 1.2.1
dask: 1.2.0
distributed: 1.27.1
matplotlib: 3.0.3
cartopy: 0.17.0
seaborn: 0.9.0
setuptools: 41.0.1
pip: 19.1
conda: None
pytest: None
IPython: 7.5.0
sphinx: None


In [3]:
# Setup Dask Cluster / Client
client = Client(processes=True, n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:51060 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 34.28 GB


creat a list of filenames, only necessary if you replicate the directory structure of the Ftp site

In [4]:
#read_date,end_date = np.datetime64('2018-04-10'),np.datetime64('2018-06-12')
#read_date,end_date = np.datetime64('2002-06-01'),np.datetime64('2019-04-01')
#filelist = []
#while read_date<=(end_date+np.timedelta64(1,'D')):
#    tem_filename,exists = get_sat_filename(read_date)
#    if exists:
#        filelist.append(tem_filename)
#    read_date=read_date+np.timedelta64(1,'D')
#print(filelist[0])
from glob import glob
filelist = glob('F:/data/sst/jpl_mur/v4.1/2018/*/*',recursive=True)
filelist[0]

'F:/data/sst/jpl_mur/v4.1/2018\\100\\20180410090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'

In [7]:
# Open collection of netCDF files, concat along the ensemble dimension
#ds = xr.open_mfdataset('/pool0/home/jhamman1/workdir/newman_ensemble/conus_ens_[01]*nc',
#                       concat_dim=xr.Variable('ensemble', np.arange(100)),
#                       chunks={'time': 366, 'lat': 224, 'lon': 464},
#                       autoclose=True)
ds = xr.open_mfdataset(filelist,coords='minimal', 
                       chunks={'time': 64, 'lat': 100, 'lon': 100})
#17999, 36000


In [8]:
display(ds)

<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 64)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2018-04-10T09:00:00 ... 2018-06-12T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<shape=(64, 17999, 36000), chunksize=(1, 100, 100)>
    analysis_error    (time, lat, lon) float32 dask.array<shape=(64, 17999, 36000), chunksize=(1, 100, 100)>
    mask              (time, lat, lon) float32 dask.array<shape=(64, 17999, 36000), chunksize=(1, 100, 100)>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<shape=(64, 17999, 36000), chunksize=(1, 100, 100)>
    dt_1km_data       (time, lat, lon) timedelta64[ns] dask.array<shape=(64, 17999, 36000), chunksize=(1, 100, 100)>
Attributes:
    Conventions:                CF-1.5
    title:                      Daily MUR SST, Fi

In [ ]:
#ds['mask'] = ds['mask'].isel(ensemble=0, drop=True)
#ds['elevation'] = ds['elevation'].isel(ensemble=0, drop=True)

In [ ]:
#display(ds)

In [9]:
# settings for compression/encoding
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}

In [ ]:
# wrtie the zarr dataset
ds.to_zarr('F:/data/sst/jpl_mur/v4.1/MUR_Saildrone2018BajaCruise.zarr', mode='w', encoding=encoding)

distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:51717 remote=tcp://127.0.0.1:51060>
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 62% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garba

In [9]:
client.close()
# Push things to google cloud storage
# !gsutil -m cp -r /pool0/home/jhamman1/workdir/gmet.zarr gs://pangeo-data/

distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing
distributed.nanny - WARNING - Worker process 48988 was killed by signal 15
distributed.core - INFO - Event loop was unresponsive in Nanny for 39.99s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nann

In [ ]:
client

some testing and imaging of MUR SST data

In [ ]:
subset = ds.sel(lon=slice(-130,-114),lat=slice(20,40))
subset.load()
subset.analysed_sst.mean({'time'}).plot()


In [ ]:
subset.analysed_sst.std({'time'}).plot()

In [ ]:
subset.analysed_sst.mean({'lat','lon'}).plot()

In [ ]:
subset.analysed_sst[0,:,:].plot()